国土基本図郭を手軽に参照するWebアプリを作成するためのメモ的なもの
by Hiromu Daimaru on 6 Nov. 2022
streamlit ではマップ上のマーカーのポップアップを表示することが出来ないため、ここではstreamlit_foliumを使用しています。
下記のサイトを参考にしました。「国土基本図の図郭」を作成する
https://qiita.com/shiba54/items/29b8722189fbfe5235cb

In [1]:
import streamlit as st                      # streamlit
from streamlit_folium import st_folium      # streamlitでfoliumを使う
import folium                               # folium
import pandas as pd
from pyproj import CRS
from pyproj import Transformer
import itertools

公共測量標準図式の第84条の4の一、二、三
区画名は、各座標系のＹ軸及びＸ軸を基準とし、南北300km、東西160km を含む区域を30km×40kmの長方形に分割して区画を定め、下図によりアルファベット大文字の組合せで表示する。

In [2]:
# 平面直角座標系（JGD2011）の EPSGコードを返す関数
def get_epsgXY_2011(kei: int) -> int:
    return kei + 6668

In [3]:
# このアプリでは最大で13系まで扱います
max_system = 13
# 系の数字
numbers = list(range(1, max_system + 1))
# 系の数字の文字
systems = [str(n) for n in numbers]    


In [4]:
# 各座標系の中心点の緯度経度の配列。13系まで。0系は無いので, [0, 0]としてある。
centers = [[0, 0], [33, 129.5], [33, 131.0], [36, 132.17], [33, 133.5], [36, 134.33], [36, 136.0], [36, 137.17], [36, 138.5], [36, 139.83], [40, 140.83], [44, 140.25], [44, 142.25], [44, 144.25]]


In [5]:
# 仮に平面直角座標系を7に設定する
# streamlit では、下記によりセレクトボックスから選択する
# selected_system = st.sidebar.selectbox('平面直角座標系',systems)
selected_system = '7'
kei = int(selected_system)


In [6]:
# pyprojによる座標変換　下記サイトを参照
# 平面直角座標系（JGD2011: EPSG6675）第7系 -> 緯度経度（WGS1984: EPSG4326）
# https://pyproj4.github.io/pyproj/stable/examples.html
crs_4326 = CRS.from_epsg(4326) # 変換先の空間参照、WGS1984の緯度経度
crs_dst = CRS.from_epsg(get_epsgXY_2011(kei)) # 変換元の空間参照、平面直角座標系 (m)
# 変換器の定義
transformer = Transformer.from_crs(crs_dst, crs_4326)

In [7]:
# 南北方向の位置を示すA-Tの文字を配列として与える
l_ycode = []
for i in range(20):
    y_code = chr(65 + i)
    #print(y_code)
    l_ycode.append(y_code)
# 東西方向の位置を示すA-Hの文字を配列として与える
l_xcode = []
for i in range(8):
    x_code = chr(65 + i)
    l_xcode.append(x_code)

In [8]:
# データフレームの準備
# 50000レベル図郭の情報を格納するデータフレームの作成
cols = ['name', 'nwcorner_x', 'nwcorner_y', 'center_x', 'center_y','lon','lat']
df = pd.DataFrame(index=[],columns=cols)

In [9]:
# 空のデータフレームが作成されたことを確認
df

,name,nwcorner_x,nwcorner_y,center_x,center_y,lon,lat


In [10]:
# 図郭範囲全体の北西端と北西端メッシュの中心座標を定義する
nw_origin_x = -160000
nw_origin_y = 300000 
nw_center_x = nw_origin_x + 20000
nw_center_y = nw_origin_y - 15000

In [11]:
for iy in range(0, 20):
    center_y = nw_center_y - 30000 * iy
    nwcorner_y = nw_origin_y - 30000 * iy
    for ix in range(0, 8):
        code = f'{kei:02}' + l_ycode[iy] + l_xcode[ix]
        center_x = nw_center_x + 40000 * ix
        nwcorner_x = nw_origin_x + 40000 * ix
        lat,lon = transformer.transform(center_y, center_x)
        new_data = [[code, nwcorner_x, nwcorner_y, center_x, center_y, lon, lat]]
        df_newrow = pd.DataFrame(new_data,columns=cols)
        # concatにより新たな行を追加
        df = pd.concat([df, df_newrow], axis=0)

In [12]:
# データフレームから各図郭の名称を取得
maps50k = df['name']
# seriesとして戻った値を配列に変換する
codes_50k = maps50k.values.tolist()
# 以下では 07IC が選択されたと仮定する
selected_50k_code = '07IC'
# streamlit では下記によりセレクトボックスから選択する
#selected_50k_code = st.sidebar.selectbox('50000図郭一覧',codes_50k)

In [13]:
# 地図の中心の緯度/経度、タイル、初期のズームサイズを指定
m = folium.Map(
    # 地図の中心位置の指定
    location= centers[kei], 
    # タイル、アトリビュートの指定
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='平面直角座標系の図郭中心',
    # ズームを指定
    zoom_start=8
)

In [14]:
# 読み込んだデータ(緯度・経度、ポップアップ用文字、アイコンを表示)
for i, row in df.iterrows():
    # ポップアップの作成(図郭名)
    pop= row['name']
    folium.Marker(
        # 緯度と経度を指定
        location=[row['lat'], row['lon']],
        # ツールチップの指定
        tooltip=row['name'],
        # ポップアップの指定
        popup=folium.Popup(pop, max_width=300),
        # アイコンの指定(アイコン、色)
        icon=folium.Icon(icon="home",icon_color="white", color="red")
    ).add_to(m)

In [15]:
# 地図の表示
m
# streamlitでは下記を使用
#st_data = st_folium(m, width=400, height=400)

In [16]:
# 指定された50000図郭の北西端の座標と中心の緯度経度を取得する
nw_origin50k_x, nw_origin50k_y = df[df['name']==selected_50k_code]['nwcorner_x'].iloc[-1], df[df['name']==selected_50k_code]['nwcorner_y'].iloc[-1]
lat_center50k, lon_center50k = df[df['name']==selected_50k_code]['lat'].iloc[-1], df[df['name']==selected_50k_code]['lon'].iloc[-1]

In [17]:
# レベル5000用のデータフレームを作成する
df5k = pd.DataFrame(index=[],columns=cols)
for iy in range(0, 10):
    nwcorner_y = nw_origin50k_y - 3000 * iy
    center_y = nwcorner_y - 1500
    for ix in range(0, 10):
        code = selected_50k_code + str(iy) + str(ix)
        nwcorner_x = nw_origin50k_x + 4000 * ix
        center_x = nwcorner_x + 2000
        lat,lon = transformer.transform(center_y, center_x)
        new_data = [[code, nwcorner_x, nwcorner_y, center_x, center_y, lon, lat]]
        df_newrow = pd.DataFrame(new_data,columns=cols)
        df5k = pd.concat([df5k, df_newrow], axis=0)

In [18]:
# レベル5000用のマップを作成する
m2 = folium.Map(
    # 地図の中心位置の指定
    location= [lat_center50k, lon_center50k], 
    # タイル、アトリビュートの指定
    #tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr= selected_50k_code,
    # ズームを指定
    zoom_start=10
)

In [19]:
# レベル5000用のマップにマーカーをプロットする
for i, row in df5k.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        icon=folium.Icon(color='red')
    ).add_to(m2)

In [20]:
# 地図の表示
m2
# streamlitでは下記を使用
#st_data = st_folium(m2, width=400, height=400)

In [21]:
# レベル5000図郭のリストを作成して、対象図郭を選択する
maps5k = df5k['name']
codes_5k = maps5k.values.tolist()
# 以下では、仮に 07IC45 を指定する
selected_5k_code = '07IC45'
# streamlitでは5000図郭のセレクトボックスから選択する
#selected_5k_code = st.sidebar.selectbox('5千図郭一覧',codes_5k)

In [22]:
# 指定された5000図郭の北西端の座標と中心の緯度経度を取得する
nw_origin5k_x, nw_origin5k_y = df5k[df5k['name']==selected_5k_code]['nwcorner_x'].iloc[-1], df5k[df5k['name']==selected_5k_code]['nwcorner_y'].iloc[-1]
lat_center5k, lon_center5k = df5k[df5k['name']==selected_5k_code]['lat'].iloc[-1], df5k[df5k['name']==selected_5k_code]['lon'].iloc[-1]


In [23]:
# ポリゴン描画用に東西南北の緯度経度の列を追加する
corner_cols = ['nw_lon', 'nw_lat', 'ne_lon', 'ne_lat', 'se_lon', 'se_lat', 'sw_lon','sw_lat']

In [24]:
#2500レベルのデータベースの作成
# 2500レベルのデータベースには
# 南東端と北西端の緯度経度のカラムを追加する
cols2500=cols + corner_cols
df2500 = pd.DataFrame(index=[],columns=cols2500)

In [25]:
df2500

,name,nwcorner_x,nwcorner_y,center_x,center_y,lon,lat,nw_lon,nw_lat,ne_lon,ne_lat,se_lon,se_lat,sw_lon,sw_lat


In [26]:
# 4分割図郭のオフセット値の定義
l_offset2500 = [[0, 0], [2000, 0], [0, -1500],[2000, -1500]]

In [27]:
# 2500レベル図郭のデータベースの構築
for i in range(0, 4):
    nwcorner_x = nw_origin5k_x + l_offset2500[i][0]
    nwcorner_y = nw_origin5k_y + l_offset2500[i][1]
    code = selected_5k_code + str(i+1)
    center_x, center_y = nwcorner_x + 1000, nwcorner_y - 750
    lat,lon = transformer.transform(center_y, center_x)
    # 東西南北単の座標を緯度経度に変換
    nw_lat, nw_lon = transformer.transform(center_y + 750, center_x - 1000)
    ne_lat, ne_lon = transformer.transform(center_y + 750, center_x + 1000)
    se_lat, se_lon = transformer.transform(center_y - 750, center_x + 1000)
    sw_lat, sw_lon = transformer.transform(center_y - 750, center_x - 1000)

    new_data = [[code, nwcorner_x, nwcorner_y, center_x, center_y, lon, lat, nw_lon, nw_lat, ne_lon, ne_lat, se_lon, se_lat, sw_lon, sw_lat]]
    df_newrow = pd.DataFrame(new_data,columns=cols2500)
    df2500 = pd.concat([df2500, df_newrow], axis=0)

In [28]:
df2500

,name,nwcorner_x,nwcorner_y,center_x,center_y,lon,lat,nw_lon,nw_lat,ne_lon,ne_lat,se_lon,se_lat,sw_lon,sw_lat
0,07IC451,-60000,48000,-59000,47250,136.508700,36.424047,136.497490,36.430744,136.519795,36.430867,136.519907,36.417349,136.497606,36.417227
0,07IC452,-58000,48000,-57000,47250,136.531002,36.424168,136.519795,36.430867,136.542100,36.430986,136.542208,36.417468,136.519907,36.417349
0,07IC453,-60000,46500,-59000,45750,136.508814,36.410529,136.497606,36.417227,136.519907,36.417349,136.520019,36.403831,136.497722,36.403709
0,07IC454,-58000,46500,-57000,45750,136.531112,36.410650,136.519907,36.417349,136.542208,36.417468,136.542316,36.403950,136.520019,36.403831


In [29]:
# 2500レベル図郭描画用のマップの設定
m3 = folium.Map(
    # 地図の中心位置の指定
    location= [lat_center5k, lon_center5k], 
    # タイル、アトリビュートの指定
    #tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr= selected_50k_code,
    # ズームを指定
    zoom_start=13
)

In [32]:
# 描画テスト
#corners = [(36.430744, 136.497490), (36.430744, 136.519907), (36.417349, 136.519907), (36.417349, 136.497490)]
nw, ne, se, sw = (36.430744, 136.497490), (36.430744, 136.519907), (36.417349, 136.519907), (36.417349, 136.497490)
corners = [nw, ne, se, sw]

folium.Polygon(
    #location= (36.424047,136.508700)
    locations= corners,
    color="red", # 線の色
    weight=10, # 線の太さ
    fill=True, # 塗りつぶす
    fill_opacity=0.5 # 透明度（1=不透明）
).add_to(m3)

In [33]:
m3

In [36]:
# 試験中
# 2500レベル用のマップに4つの矩形を描画する

for i, row in df2500.iterrows():
    nw, ne, se, sw = (row['nw_lat'], row['nw_lon']), (row['ne_lat'], row['ne_lon']), (row['se_lat'], row['se_lon']), (row['sw_lat'], row['sw_lon'])
    corners = [nw, ne, se, sw]
    folium.Polygon(
        locations = corners,
        popup = row['name'],
        color="red", # 線の色
        weight=10, # 線の太さ
        fill=True, # 塗りつぶす
        fill_opacity=0.5 # 透明度（1=不透明）
    ).add_to(m3)

In [37]:
m3

In [27]:
# 2500レベル用のマップに4つの矩形を描画する
for i, row in df2500.iterrows():
    folium.Polygon(
        
        location=[se_lat, se_lon,],
        popup=row['name'],
        icon=folium.Icon(color='red')
    ).add_to(m3)

In [27]:
# 2500レベル用のマップにマーカーをプロットする
for i, row in df2500.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        icon=folium.Icon(color='red')
    ).add_to(m3)

In [28]:
# 地図の表示
m3
# streamlitでは下記を使用
#st_data = st_folium(m3, width=400, height=400)